In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('correct_for_learning.csv')
df = df[['post_text', 'post_date', 'user_id', 'emo', 'soc',
       'psych', 'health', 'emo_class', 'soc_class', 'psych_class',
       'health_class']]
df.columns = ['text', 'post_date', 'user_id', 'emo', 'soc', 'psych', 'health',
       'emo_class', 'soc_class', 'psych_class', 'health_class']

In [ ]:
from transformers import pipeline


In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

model_name = "DeepPavlov/rubert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_analysis = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device='cuda:0'
)

'from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer\n\nmodel_name = "DeepPavlov/rubert-base-cased"\nmodel = AutoModelForSequenceClassification.from_pretrained(model_name)\ntokenizer = AutoTokenizer.from_pretrained(model_name)\nsentiment_analysis = pipeline(\n    "sentiment-analysis",\n    model=model,\n    tokenizer=tokenizer,\n    device=\'cuda:0\'  \n)'

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment', return_dict=True)
model.to(device)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    inputs.to(device)
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted


In [ ]:
texts = ['какой чудесный день! я счастлив', 'я себя ненавижу это место отвратительно и меня бесит', 'в окне пролетела птица']

predict(texts)

array([1, 2, 0])

In [ ]:
results = []
for text in list(df.text.tolist()):
  pred = predict(text)[0]
  results.append(pred)

In [ ]:
df['sentiment'] = results

In [ ]:
df

,text,post_date,user_id,emo,soc,psych,health,emo_class,soc_class,psych_class,health_class,sentiment
0,Я смеялся до слёз. Пародия про каникулы презид...,2020,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,1
1,"Торт ""Наполеон"" на сковороде Готовится, быстро...",2020,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,1
2,"Сольный концерт Софии Ротару в Кремле ( т/к ""И...",2013,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,0
3,TINA TURNER & STING - ON SILENT WINGS (2),2013,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,0
4,Tina Turner - Something Beautiful Remains (Pro...,2013,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
152666,Все,2010,7329,1.000000,2.8,2.166667,2.142857,0,1,0,0,0
152667,нет пока ещё:),2008,7329,1.000000,2.8,2.166667,2.142857,0,1,0,0,0
152668,Да у меня баланс умер:(А пополнить пока некому:),2008,7329,1.000000,2.8,2.166667,2.142857,0,1,0,0,2
152669,Спаси Господи.В инет очень давно уже не выходи...,2008,7329,1.000000,2.8,2.166667,2.142857,0,1,0,0,0


In [ ]:
def name_match(x):
  if x == 0:
    return 'neutral'
  if x == 1:
    return 'positive'
  if x == 2:
    return 'negative'

In [ ]:
df['sent_name'] = df.sentiment.apply(name_match)
df

,text,post_date,user_id,emo,soc,psych,health,emo_class,soc_class,psych_class,health_class,sentiment,sent_name
0,Я смеялся до слёз. Пародия про каникулы презид...,2020,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,1,positive
1,"Торт ""Наполеон"" на сковороде Готовится, быстро...",2020,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,1,positive
2,"Сольный концерт Софии Ротару в Кремле ( т/к ""И...",2013,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,0,neutral
3,TINA TURNER & STING - ON SILENT WINGS (2),2013,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,0,neutral
4,Tina Turner - Something Beautiful Remains (Pro...,2013,856,3.666667,0.8,2.833333,2.285714,1,0,1,0,0,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152666,Все,2010,7329,1.000000,2.8,2.166667,2.142857,0,1,0,0,0,neutral
152667,нет пока ещё:),2008,7329,1.000000,2.8,2.166667,2.142857,0,1,0,0,0,neutral
152668,Да у меня баланс умер:(А пополнить пока некому:),2008,7329,1.000000,2.8,2.166667,2.142857,0,1,0,0,2,negative
152669,Спаси Господи.В инет очень давно уже не выходи...,2008,7329,1.000000,2.8,2.166667,2.142857,0,1,0,0,0,neutral


In [ ]:
df.to_csv('learning_features.csv', index = False)

In [ ]:
!pip install --upgrade spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 67.9 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


In [ ]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 63.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=6059c23ef2c1b3952ab1e6f0251fd824254649e03d4d2d0015f84aac62f36a10
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# pip install -U spacy
import spacy
from spacy import displacy
from IPython.core.display import display, HTML
# !python -m spacy download ru_core_news_lg

In [ ]:
from razdel import sentenize